In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14831,2024-04-14,Japão B2.League,02:00,Shiga,Kobe,1.47,2.56,162.5,1.80,1.86,0.0,0.00,0.00,IFZsO056,0.680272,0.390625,0.555556,0.537634,0.070897,122.326,24.902146,0.203572,3.0,0.000000,0.000000,120.54,1.420,0.183984,0.129566,73.0,139.560,33.889718,0.242833,1.2,1.643168,1.369306,186.88,1.732,0.470925,0.271896,1.0,82,73,1.47,2.56,100.080,174.804,0.382504,0.023184,NaN,1.28,0.256,1.835937,0.791556,1.0,0.208444,1.51,0.302,5.165563,0.506194,0.5,-0.006194
14832,2024-04-14,Macedônia Do Norte Prva Liga,13:00,Euro Nickel,Pelister,2.13,1.66,160.5,1.85,1.85,1.5,2.00,1.51,nJkSfhHL,0.469484,0.602410,0.540541,0.540541,0.071893,168.922,100.588267,0.595472,0.6,1.341641,2.236068,328.10,2.336,1.236155,0.529176,-54.0,452.360,756.720158,1.672827,3.0,0.000000,0.000000,110.70,5.190,8.838317,1.702951,67.0,85,90,3.86,1.23,134.732,466.560,0.175377,0.000000,0.197426,-2.57,-0.514,-2.198444,0.000000,0.0,0.000000,21.34,4.268,0.154639,0.000000,0.0,0.000000
14833,2024-04-14,Lituânia Lkl,13:30,Mazeikiai,Jonava,1.37,2.93,165.5,1.86,1.84,0.0,0.00,0.00,KnzrUiD0,0.729927,0.341297,0.537634,0.543478,0.071224,247.590,116.241605,0.469492,1.2,1.643168,1.369306,396.24,3.082,1.434528,0.465454,-21.0,232.936,83.907746,0.360218,1.2,1.643168,1.369306,180.96,2.960,1.054680,0.356311,-17.0,78,78,5.08,2.32,191.138,477.456,0.513064,0.007644,NaN,3.48,0.696,0.531609,0.468662,0.6,0.131338,4.70,0.940,2.053191,0.000000,0.0,0.000000
14834,2024-04-14,Lituânia Lkl,11:20,Zalgiris Kaunas,Rytas,1.26,3.61,165.5,1.87,1.79,0.0,0.00,0.00,vmsfRkcJ,0.793651,0.277008,0.534759,0.558659,0.070659,107.700,8.817789,0.081874,3.0,0.000000,0.000000,107.90,1.164,0.114368,0.098254,72.0,131.572,29.323071,0.222867,2.4,1.341641,0.559017,154.81,1.388,0.234883,0.169224,49.0,83,113,1.30,1.37,109.354,171.926,0.682423,0.030912,NaN,0.75,0.150,1.733333,0.000000,0.0,0.000000,0.73,0.146,17.876712,0.000000,0.0,0.000000
14835,2024-04-14,Japão B2.League,02:10,Niigata,Fukushima,3.53,1.27,167.5,1.85,1.85,0.0,0.00,0.00,4CVwPtj0,0.283286,0.787402,0.540541,0.540541,0.070688,318.596,88.244680,0.276980,0.0,0.000000,NaN,282.58,4.638,1.226487,0.264443,-88.0,349.462,223.163204,0.638591,0.6,1.341641,2.236068,109.80,4.184,2.679520,0.640421,-37.0,71,90,3.98,1.22,402.496,316.508,0.665859,0.000000,NaN,0.77,0.154,16.428571,0.308774,0.1,-0.208774,-2.48,-0.496,-0.544355,0.449175,0.5,0.050825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15025,2024-04-14,Espanha Leb Ouro,15:00,Castello,Hestia Menorca,1.70,2.05,156.5,1.85,1.85,-2.5,1.95,2.48,AFid5cGi,0.588235,0.487805,0.540541,0.540541,0.076040,311.622,166.658938,0.534811,1.2,1.643168,1.369306,448.23,4.140,2.375163,0.573711,-74.0,187.050,141.886988,0.758551,1.8,1.643168,0.912871,182.16,2.530,1.932912,0.763997,-4.0,67,92,6.69,1.98,179.512,272.912,0.131993,0.000000,0.169195,-3.59,-0.718,-0.974930,0.439257,0.2,-0.239257,-3.02,-0.604,-1.738411,0.316514,0.4,0.083486
15026,2024-04-14,Espanha Liga Feminina,07:00,Jairis F,Cadi La Seu F,1.14,5.24,135.5,1.83,1.83,0.0,0.00,0.00,8x66wX37,0.877193,0.190840,0.546448,0.546448,0.068033,128.076,94.928816,0.741191,2.4,1.341641,0.559017,71.98,1.844,1.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
14871,02:05,Japão B.League,Osaka,Kyoto,2.05,Back Home
14919,11:00,Suécia Superettan,Huddinge,Sloga Uppsala,1.80,Back Home
14926,03:30,Taiwan P. League+,Formosa Dreamers,Taoyuan Pauian Pilots,1.88,Back Home
14932,15:00,Itália Liga A,Scafati,Varese,1.61,Back Home
14942,20:30,Argentina Liga A,Penarol,San Martin,1.86,Back Home
14970,16:30,Eua Nba,Los Angeles Clippers,Houston Rockets,2.41,Back Home
14972,16:30,Eua Nba,Minnesota Timberwolves,Phoenix Suns,1.79,Back Home
14993,11:15,Grécia Liga De Basquete,AEK Athens,Lavrio,1.75,Back Home
